## OOI Machine to Machine (M2M) Realtime Requests

### Getting Started
* Create a user account on ooinet.oceanobservatories.org
* Log in
* Navigate to the drop down menu screen in the top-right corner menu
* Click on the "User Profile" element of the drop down.
* Copy and save the following data from the user profile: API Username and API Token. The API Username is similar to “OOIAPI-QTULEV9STCAS55”. The API Token is similar to “YXP2Q2W4SOP”.

In [ ]:
# specify your username and token
username = 'API Username'
token = 'API Token'

In this example we will request pressure data from the Cabled Axial Base (RS03AXPS) - Shallow Profiler (SF03A) - CTD (2A-CTDPFA302) sensor.

Reference Designator: RS03AXPS-SF03A-2A-CTDPFA302  
Delivery Method: streamed  
Stream: ctdpf_sbe43_sample  
Parameter: seawater_pressure  

A good resource for finding the information you will need to input below is the Data Team Portal at http://ooi.visualocean.net/

In [43]:
# specify your inputs
sub_site = 'RS03AXPS'
platform = 'SF03A'
instrument = '2A-CTDPFA302'
delivery_method = 'streamed'
stream = 'ctdpf_sbe43_sample'
parameter = 'seawater_pressure'

In [44]:
import datetime
import time
import requests
import pprint
from concurrent.futures import ThreadPoolExecutor

In [38]:
# setup
BASE_URL = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'
pool = ThreadPoolExecutor(1)
ntp_epoch = datetime.datetime(1900, 1, 1)
unix_epoch = datetime.datetime(1970, 1, 1)
ntp_ordinal = ntp_epoch.toordinal()
ntp_delta = (unix_epoch - ntp_epoch).total_seconds()

In [39]:
# basic functions
def ntp_seconds_to_datetime(ntp_seconds):
    return datetime.datetime.utcfromtimestamp(ntp_seconds - ntp_delta).replace(microsecond=0)

def get_future_data(url, params, username, token):
    auth = (username, token)
    return pool.submit(requests.get, url, params=params, auth=auth)

def extract_keys(data, keys, min_time):
    rdict = {key: [] for key in keys}
    for record in data:
        if record['time'] <= min_time:
            time_r = record['time']
            time_r = ntp_seconds_to_datetime(time_r)
            time_r = time_r.strftime("%Y-%m-%d %H:%M:%S.000Z")
            print 'No new data found since ' + str(time_r) + '. Sending new request.'
            continue
        for key in keys:
            rdict[key].append(record[key])
    print 'Found %d new data points after filtering' % len(rdict['time'])
    return rdict


The following script is the main script. It will execute 10 requests, but if put into a while loop it can execute data requests indefinitely. See comments for explanations in line. The data response in json format contains all parameters under the input data stream, however, only prints out the values for the input parameter and corresponding timestamp.

In [63]:
def requestNow(username, token, sub_site, platform, instrument, delivery_method, stream, parameter):
    
    # create the base url
    request_url = '/'.join((BASE_URL, sub_site, platform, instrument, delivery_method, stream))
    
    # specify parameters which will be used in the get_future_data function. with each new request
    # being sent, only the beginDT will change. it will be set to the time stamp of the last data point received.
    # notice that there is no endDT specified, as a request with a beginDT and no endDT will return everything 
    # from beginDT until present, up to 1000 data points.
    params = {
        'beginDT': None,
        'limit': 1000,
        'user': 'realtime',
    }
    
    # start with the last 10 seconds of data from present time
    begin_time = datetime.datetime.utcnow() - datetime.timedelta(seconds=10)
    
    # last_time will be assigned as the time stamp of the last data point received once the first
    # request is sent
    last_time = 0
    
    for i in range(10): # replace with `while True:` to enter an endless data request loop
        
        parameter_values = []
        time_values = []
        
        # update beginDT for this request
        begin_time_str = begin_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
        params['beginDT'] = begin_time_str

        # send request in thread
        data_future = get_future_data(request_url, params, username, token)
        
        # poll until complete
        while not data_future.done:
            # while request not complete, yield control to event loop
            time.sleep(0.1)

        # request complete, if not 200, log error and try again
        response = data_future.result()
        if response.status_code != 200:
            print 'Error fetching data', response.text
            time.sleep(0.1)
            continue
        
        # store json response
        data = response.json()
        
        # use extract_keys function to inform users about whether or not data is being returned. 
        # parse data in json response for input parameter and corresponding timestamp
        data = extract_keys(data, ['time', parameter], min_time=last_time)

        # if no data is returned, try again
        if not data['time']:
            time.sleep(0.1)
            continue

        # set beginDT to time stamp of last data point returned
        last_time = data['time'][-1]
        begin_time = ntp_seconds_to_datetime(last_time)

        # print data points returned
        print "\n"
        pprint.pprint(data)
        print "\n"
        

In [64]:
requestNow(username, token, sub_site, platform, instrument, delivery_method, stream, parameter)

Found 7 new data points after filtering


{'seawater_pressure': [47.90870860074877,
                       47.88517629075554,
                       47.85736538556815,
                       47.83062373315278,
                       47.812440092344396,
                       47.80602219484076,
                       47.811370037570484],
 'time': [3709487730.1028037,
          3709487731.102813,
          3709487732.1029267,
          3709487733.1032495,
          3709487734.1032586,
          3709487735.102853,
          3709487736.102863]}


No new data found since 2017-07-19 21:15:36.000Z. Sending new request.
Found 0 new data points after filtering
No new data found since 2017-07-19 21:15:36.000Z. Sending new request.
Found 0 new data points after filtering
No new data found since 2017-07-19 21:15:36.000Z. Sending new request.
Found 5 new data points after filtering


{'seawater_pressure': [47.8263451338373,
                       47.84880778213796,
                       47.8712708